# Predicting Flights Delay: Flights Feature Engineering

In [147]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [148]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [149]:
flights = pd.read_csv('../data/flights_data_merge.csv')

In [150]:
def extract_features_df(df):
    
    '''
    Extract relevant features from flight dataframe.
        Input: Raw dataframe
        Output: Dataframe with relevant features for analysis
    '''
    
    new_df = df.drop(['branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num',
                      'op_carrier_fl_num', 'cancellation_code', 'carrier_delay', 'weather_delay', 
                      'nas_delay', 'security_delay', 'late_aircraft_delay', 'first_dep_time',
                      'total_add_gtime', 'longest_add_gtime', 'no_name', 'cancelled', 'diverted', 'taxi_out', 
                      'wheels_off', 'wheels_on', 'taxi_in', 'dup', 'dep_time', 'arr_time', 'actual_elapsed_time', 'air_time'], axis=1).dropna()
    
    new_df['fl_date'] = pd.to_datetime(new_df['fl_date'], format='%Y-%m-%d')

    return new_df

In [151]:
flights = extract_features_df(flights)
flights.drop('Unnamed: 0', axis=1, inplace=True)

In [152]:
flights.drop('flights', axis=1, inplace=True)

In [153]:
flights.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,arr_delay,crs_elapsed_time,distance
0,2018-03-31,UA,UA,N76288,11618,EWR,"Newark, NJ",11697,FLL,"Fort Lauderdale, FL",1705,46.0,2017,38.0,192.0,1065
1,2018-11-27,DL,DL,N902DN,11884,GEG,"Spokane, WA",14869,SLC,"Salt Lake City, UT",544,1.0,847,-10.0,123.0,546
2,2018-09-17,AA,YV,N912FJ,11298,DFW,"Dallas/Fort Worth, TX",15624,VPS,"Valparaiso, FL",1650,54.0,1846,47.0,116.0,641
3,2018-10-19,G4,G4,252NV,10135,ABE,"Allentown/Bethlehem/Easton, PA",13577,MYR,"Myrtle Beach, SC",1108,656.0,1239,654.0,91.0,518
4,2018-04-24,DL,EV,N738EV,11042,CLE,"Cleveland, OH",12953,LGA,"New York, NY",912,-10.0,1048,-8.0,96.0,419


In [154]:
flights.shape

(980258, 16)

### Feature 1: Analysis of Dates

<b> 1. Separate the date into months and days </b>

In [155]:
flights['month'] = flights['fl_date'].dt.month
flights['day'] = flights['fl_date'].dt.day
flights.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,arr_delay,crs_elapsed_time,distance,month,day
0,2018-03-31,UA,UA,N76288,11618,EWR,"Newark, NJ",11697,FLL,"Fort Lauderdale, FL",1705,46.0,2017,38.0,192.0,1065,3,31
1,2018-11-27,DL,DL,N902DN,11884,GEG,"Spokane, WA",14869,SLC,"Salt Lake City, UT",544,1.0,847,-10.0,123.0,546,11,27
2,2018-09-17,AA,YV,N912FJ,11298,DFW,"Dallas/Fort Worth, TX",15624,VPS,"Valparaiso, FL",1650,54.0,1846,47.0,116.0,641,9,17
3,2018-10-19,G4,G4,252NV,10135,ABE,"Allentown/Bethlehem/Easton, PA",13577,MYR,"Myrtle Beach, SC",1108,656.0,1239,654.0,91.0,518,10,19
4,2018-04-24,DL,EV,N738EV,11042,CLE,"Cleveland, OH",12953,LGA,"New York, NY",912,-10.0,1048,-8.0,96.0,419,4,24


<b> 2. Obtain median arrival and departure delays for each month; categorize dates into weekend and weekday </b>

In [157]:
monthly_delays = flights.groupby('month')[['arr_delay', 'dep_delay']].median().reset_index().rename(columns={'arr_delay': 'month_arr_delay_avg', 'dep_delay':'month_dep_delay_avg'})

In [158]:
flights = pd.merge(flights, monthly_delays, on='month', how='inner')

In [160]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'op_unique_carrier', 'tail_num',
       'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id',
       'dest', 'dest_city_name', 'crs_dep_time', 'dep_delay', 'crs_arr_time',
       'arr_delay', 'crs_elapsed_time', 'distance', 'month', 'day',
       'month_arr_delay_avg', 'month_dep_delay_avg'],
      dtype='object')

In [161]:
def is_weekend(date):
    weekno = date.weekday()
    
    if weekno < 5:
        return 0
    else:
        return 1

In [162]:
flights['is_weekend'] = flights['fl_date'].map(is_weekend)

<b> 3. Convert time to datetime objects, separate into hours. </b>

In [167]:
def convert_time(time):
    
    if time == 2400:
        return '23:59'
    
    time_string = str(int(time))
    
    if len(time_string) < 4:
        for _ in range(4-len(time_string)):
            time_string = '0' + time_string
    
    return time_string[:2] + ':' + time_string[2:]

In [168]:
flights['crs_dep_time'] = flights['crs_dep_time'].map(convert_time)
flights['crs_arr_time'] = flights['crs_arr_time'].map(convert_time)

In [169]:
flights['crs_dep_time'] = pd.to_datetime(flights['crs_dep_time'], format='%H:%M')
flights['crs_arr_time'] = pd.to_datetime(flights['crs_arr_time'], format='%H:%M')

In [170]:
flights['dep_hour'] = flights['crs_dep_time'].dt.hour
flights['dep_min'] = flights['crs_dep_time'].dt.minute

In [171]:
flights['arr_hour'] = flights['crs_arr_time'].dt.hour
flights['arr_min'] = flights['crs_arr_time'].dt.minute

In [172]:
flights.drop(['crs_dep_time', 'crs_arr_time'], axis=1, inplace=True)

<b> 4. Categorize arrival and departure times </b>

In [173]:
def get_time_desc(hour):
    if hour == 0:
        return 'midnight'
    elif hour > 0 and hour <= 4:
        return 'early morning'
    elif hour > 4 and hour <= 10:
        return 'morning'
    elif hour > 10 and hour <= 15:
        return 'noon'
    elif hour > 15 and hour <= 21:
        return 'evening'
    else:
        return 'late night'

In [174]:
flights['dep_time_desc'] = flights['dep_hour'].map(get_time_desc)
flights['arr_time_desc'] = flights['arr_hour'].map(get_time_desc)

### Feature 2: Tail Number Analysis

<b> 1. The average arrival delays amongst the different tail numbers are different, therefore they can be taken into account in a feature called plane quality. </b>

In [175]:
def get_plane_score(df):
    plane_quality = df.groupby('tail_num')[['arr_delay']].median().reset_index().rename(columns={'arr_delay': 'plane_score'})
    return pd.merge(df, plane_quality, on='tail_num', how='inner')

In [176]:
flights = get_plane_score(flights)

### Feature 3: Destination and Arrival Airport Analysis

<b> 1. Get airport busyness. It can be hypothesized that busy airports with a lot of flights can face more delays. </b>

In [207]:
high_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[0:6]['origin'].tolist()
medhigh_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[6:25]['origin'].tolist()
medium_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[25:117]['origin'].tolist()
low_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[117:]['origin'].tolist()

def get_origin_busy(code):
    if code in high_busy_dep:
        return 'high'
    elif code in medhigh_busy_dep:
        return 'medium-high'
    elif code in medium_busy_dep:
        return 'medium'
    else:
        return 'low'
    
high_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[0:6]['dest'].tolist()
medhigh_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[6:25]['dest'].tolist()
medium_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[25:118]['dest'].tolist()
low_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[118:]['dest'].tolist()

def get_dest_busy(code):
    if code in high_busy_dest:
        return 'high'
    elif code in medhigh_busy_dest:
        return 'medium-high'
    elif code in medium_busy_dest:
        return 'medium'
    else:
        return 'low'


In [215]:
flights['orr_busy'] = flights['origin'].map(get_origin_busy)
flights['arr_busy'] = flights['dest'].map(get_dest_busy)

<b> 2. Analyze Routes! Let's take a look at a combination of airline code, origin, destination, and month and obtain the following features: median, mean, 25%, and 75% of arrival delays. </b>

In [227]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'op_unique_carrier', 'tail_num',
       'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id',
       'dest', 'dest_city_name', 'dep_delay', 'arr_delay', 'crs_elapsed_time',
       'distance', 'month', 'day', 'month_arr_delay_avg',
       'month_dep_delay_avg', 'is_weekend', 'dep_hour', 'dep_min', 'arr_hour',
       'arr_min', 'dep_time_desc', 'arr_time_desc', 'plane_score', 'orr_busy',
       'arr_busy'],
      dtype='object')

In [237]:
flights['route'] = flights['op_unique_carrier'] + flights['origin'] + flights['dest'] + flights['month'].apply(lambda x: str(x))

In [248]:
route_mean = flights.groupby('route')[['arr_delay']].mean().reset_index().rename(columns={'arr_delay': 'route_mean_delay'})
route_median = flights.groupby('route')[['arr_delay']].median().reset_index().rename(columns={'arr_delay': 'route_median_delay'})
route_lq = flights.groupby('route')[['arr_delay']].quantile(0.25).reset_index().rename(columns={'arr_delay': 'route_lq_delay'})
route_uq = flights.groupby('route')[['arr_delay']].quantile(0.75).reset_index().rename(columns={'arr_delay': 'route_uq_delay'})

In [250]:
route_info = pd.merge(route_mean, route_median)
route_info = pd.merge(route_info, route_lq)
route_info = pd.merge(route_info, route_uq)

In [254]:
flights.shape

(980258, 29)

In [255]:
flights = pd.merge(flights, route_info, on='route', how='inner')

In [256]:
flights.shape

(980258, 33)

In [259]:
flights.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,...,dep_time_desc,arr_time_desc,plane_score,orr_busy,arr_busy,route,route_mean_delay,route_median_delay,route_lq_delay,route_uq_delay
0,2018-03-31,UA,UA,N76288,11618,EWR,"Newark, NJ",11697,FLL,"Fort Lauderdale, FL",...,evening,evening,-7.0,medium-high,medium-high,UAEWRFLL3,0.24,-11.0,-18.0,4.0
1,2019-03-07,UA,UA,N37468,11618,EWR,"Newark, NJ",11697,FLL,"Fort Lauderdale, FL",...,evening,late night,-8.0,medium-high,medium-high,UAEWRFLL3,0.24,-11.0,-18.0,4.0
2,2018-03-02,UA,UA,N68823,11618,EWR,"Newark, NJ",11697,FLL,"Fort Lauderdale, FL",...,noon,evening,-8.0,medium-high,medium-high,UAEWRFLL3,0.24,-11.0,-18.0,4.0
3,2019-03-20,UA,UA,N68823,11618,EWR,"Newark, NJ",11697,FLL,"Fort Lauderdale, FL",...,morning,noon,-8.0,medium-high,medium-high,UAEWRFLL3,0.24,-11.0,-18.0,4.0
4,2019-03-11,UA,UA,N39450,11618,EWR,"Newark, NJ",11697,FLL,"Fort Lauderdale, FL",...,evening,late night,-5.0,medium-high,medium-high,UAEWRFLL3,0.24,-11.0,-18.0,4.0


<b> 3. Let's obtain the probability of getting a delay greater than 10 minutes out of all the total flights. </b>

In [285]:
delayed_flights = flights[flights['arr_delay'] >= 10].groupby('route').size().reset_index().rename(columns={0: 'num_delayed_routes'})

In [286]:
total_flights = flights.groupby('route').size().reset_index().rename(columns={0: 'total_routes'})

In [287]:
all_flights = pd.merge(total_flights, delayed_flights, on='route', how='left')

In [289]:
all_flights['route_prob_delay'] = all_flights['num_delayed_routes']/all_flights['total_routes']

In [290]:
all_flights.fillna(0, inplace=True)

In [291]:
all_flights.head()

,route,total_routes,num_delayed_routes,route_prob_delay
0,9EABEATL1,3,0.0,0.000000
1,9EABEATL10,6,1.0,0.166667
2,9EABEATL11,8,3.0,0.375000
3,9EABEATL12,6,2.0,0.333333
4,9EABEATL2,3,1.0,0.333333


In [292]:
flights = pd.merge(flights, all_flights, on='route', how='inner')

<b> 4. Get the mean, median, 25% and 75% of historical departure delays for each route! </b>

In [296]:
route_mean_dep = flights.groupby('route')[['dep_delay']].mean().reset_index().rename(columns={'dep_delay': 'route_mean_dep_delay'})
route_median_dep = flights.groupby('route')[['dep_delay']].median().reset_index().rename(columns={'dep_delay': 'route_median_dep_delay'})
route_lq_dep = flights.groupby('route')[['dep_delay']].quantile(0.25).reset_index().rename(columns={'dep_delay': 'route_lq_dep_delay'})
route_uq_dep = flights.groupby('route')[['dep_delay']].quantile(0.75).reset_index().rename(columns={'dep_delay': 'route_uq_dep_delay'})

In [297]:
route_info_dep = pd.merge(route_mean_dep, route_median_dep)
route_info_dep = pd.merge(route_info_dep, route_lq_dep)
route_info_dep = pd.merge(route_info_dep, route_uq_dep)

In [298]:
flights = pd.merge(flights, route_info_dep, on='route', how='inner')

In [299]:
flights.shape

(980258, 40)

# That's all the features so far! Let's export the file to a csv!

In [301]:
flights.to_csv('../data/data-feature/flights-feature-edit.csv')